# ResNet kvantizácia

Podľa: https://nervanasystems.github.io/distiller/prepare_model_quant.html

1. Replace direct tensor operations with modules

* Replace re-used modules with dedicated instances

* Replace torch.nn.functional calls with equivalent modules

* Special cases - replace modules that aren't quantize-able with quantize-able variants


In [4]:
import torch
import torch.nn as nn

In [5]:
model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet18', pretrained=True)
model

Using cache found in /home/bohumil/.cache/torch/hub/pytorch_vision_v0.5.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Príprava pre dataset

In [6]:
import distiller
#import distiller.apputils.data_loaders as loader

#imagenet_path = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet"
#test_dataset = loader.imagenet_get_datasets(imagenet_path,False,True);

#dataset = loader.load_data('imagenet',imagenet_path,256,2);

RuntimeError: class '__torch__.torchvision.models.detection._utils.BalancedPositiveNegativeSampler' already defined. (register_type at /pytorch/torch/csrc/jit/script/compilation_unit.h:166)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x33 (0x7fac1dbda813 in /home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x5a47a9 (0x7fac65e527a9 in /home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/lib/libtorch_python.so)
frame #2: <unknown function> + 0x58c0be (0x7fac65e3a0be in /home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/lib/libtorch_python.so)
frame #3: <unknown function> + 0x58d393 (0x7fac65e3b393 in /home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/torch/lib/libtorch_python.so)
frame #4: <unknown function> + 0x210ba4 (0x7fac65abeba4 in /home/bohumil/FIIT/BP/BP/Zdroje_kod/dist_env/lib/python3.7/site-packages/tor

In [16]:
import torchvision,torch
#torchvision.datasets.imagenet.parse_devkit('/home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet.tar')
#imagenet_data = torchvision.datasets.ImageNet('/home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet')
#data_loader = torch.utils.data.DataLoader(imagenet_da  ta,4,True,args.nThreads)
train_dataset, test_dataset = loader.imagenet_get_datasets(imagenet_path)

In [12]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
def eval(model, dataset):
    for img in dataset.imgs:
        pred = model(img)
        if pred == 

SyntaxError: invalid syntax (<ipython-input-17-90e79f8001a9>, line 2)

In [ ]:
if torch.cuda.is_available():
    model(dataset+"/val/iamges/")
        

# Quantization
## 1. Replace direct tensor operations with modules

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BasicModule(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size):
        super(BasicModule, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        # (1) Overloaded tensor addition operation
        # Alternatively, could be called via a tensor function: skip_1.add_(identity)
        out += identity
        # (2) Relu module re-used
        out = self.relu(out)

        # (3) Using operation from 'torch' namespace
        out = torch.cat([identity, out], dim=1)
        # (4) Using function from torch.nn.functional
        out = F.sigmoid(out)

        return out 

## 2. Replace re-used modules with dedicated instances